## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/kaggle/input/h2oai-predict-the-llm/train.csv")

In [3]:
df.head()

,Question,Response,target
0,Explain the concept of coevolution.,Coevolution is a biological process that occur...,3
1,Is it possible that recurring fever and chills...,"Yes, recurring fever and chills can be a sympt...",4
2,Evaluate the expression 3!,The expression 3! represents the factorial of ...,1
3,What are the roles of different types of RNA i...,1. Messenger RNA (mRNA): mRNA carries genetic ...,3
4,What is the role of gene flow in population ge...,Gene flow refers to the movement of individual...,3


In [4]:
df.isnull().sum()

Question    0
Response    7
target      0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df.shape

(3969, 3)

In [7]:
from transformers import AutoTokenizer, AutoModel
import torch

def generate_embeddings(sentence):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
    model = AutoModel.from_pretrained("bert-base-cased", output_hidden_states=True)
    
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)

    # Extract the hidden states
    hidden_states = outputs.hidden_states

    # For word embeddings, you can typically use the output from the last layer
    word_embedding = hidden_states[-1]

    # Average the embeddings for all tokens to get a single sentence embedding
    sentence_embedding = torch.mean(word_embedding, dim=1)

    return sentence_embedding

In [8]:
'''
def generate_embeddings(sentence):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
    model = AutoModel.from_pretrained("bert-base-cased", output_hidden_states=True)
    
    word_embedding = tokenizer.encode_plus(sentence, tokenizer, model)
     
    return word_embedding 

In [9]:
new_df=pd.DataFrame()

In [ ]:
new_df['tokenized_Question']=df['Question'].apply(generate_embeddings)
new_df['tokenized_Response']=df['Response'].apply(generate_embeddings)
new_df['target']=df['target']

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
import spacy
import string
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [9]:
def spacy_tokenizer(sentence):
    if type(sentence) == float:
        sentence=sentence
    else:
        doc = nlp(sentence)
        mytokens = [ word.lemma_.lower().strip() for word in doc ]
        mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
        sentence = " ".join(mytokens)
    return sentence

In [10]:
df.head()

,Question,Response,target
0,Explain the concept of coevolution.,Coevolution is a biological process that occur...,3
1,Is it possible that recurring fever and chills...,"Yes, recurring fever and chills can be a sympt...",4
2,Evaluate the expression 3!,The expression 3! represents the factorial of ...,1
3,What are the roles of different types of RNA i...,1. Messenger RNA (mRNA): mRNA carries genetic ...,3
4,What is the role of gene flow in population ge...,Gene flow refers to the movement of individual...,3


In [11]:
new_df=pd.DataFrame()

new_df['tokenized_Question']=df['Question'].apply(spacy_tokenizer)
new_df['tokenized_Response']=df['Response'].apply(spacy_tokenizer)
new_df['target']=df['target']

In [12]:
new_df.head()

,tokenized_Question,tokenized_Response,target
0,explain concept coevolution,coevolution biological process occur specie ev...,3
1,possible recur fever chill indicate malaria,yes recur fever chill symptom malaria especial...,4
2,evaluate expression 3,expression 3 represent factorial 3 product pos...,1
3,role different type rna cell,1 messenger rna mrna ): mrna carry genetic inf...,3
4,role gene flow population genetic,gene flow refer movement individual population...,3


In [13]:
new_df.columns

Index(['tokenized_Question', 'tokenized_Response', 'target'], dtype='object')

In [14]:
X=new_df.iloc[:, 0:2]
y=new_df.iloc[:, 2]

In [15]:
X.head()

,tokenized_Question,tokenized_Response
0,explain concept coevolution,coevolution biological process occur specie ev...
1,possible recur fever chill indicate malaria,yes recur fever chill symptom malaria especial...
2,evaluate expression 3,expression 3 represent factorial 3 product pos...
3,role different type rna cell,1 messenger rna mrna ): mrna carry genetic inf...
4,role gene flow population genetic,gene flow refer movement individual population...


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec1 = TfidfVectorizer()
vec2 = TfidfVectorizer()

X_vec1 = vec1.fit_transform(X.iloc[:,0])
X_vec2 = vec2.fit_transform(X.iloc[:,1])

In [17]:
from scipy.sparse import hstack
X_vec = hstack((X_vec1, X_vec2))

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_vec,y,test_size=0.1,stratify=y)

In [19]:
import lightgbm as ltb

model = ltb.LGBMClassifier()
model.fit(X_train, y_train)

LGBMClassifier()

In [20]:
y_predict=model.predict(X_test)

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.23      0.23      0.23        56
           1       0.38      0.45      0.41        47
           2       0.12      0.11      0.12        61
           3       0.58      0.62      0.60        53
           4       0.37      0.27      0.31        78
           5       0.37      0.39      0.38        54
           6       0.26      0.31      0.29        48

    accuracy                           0.33       397
   macro avg       0.33      0.34      0.33       397
weighted avg       0.33      0.33      0.33       397



In [45]:
import pandas

In [46]:
test_df = pd.read_csv('/kaggle/input/h2oai-predict-the-llm/test.csv')
test_df.head()

,id,Question,Response
0,0,Multiply 13 by 7.,Multiplying 13 by 7 gives a result of 64.\nThe...
1,1,Who were the main protagonists in the Hundred ...,The most important characters in this war incl...
2,2,What was the importance of the Truman Doctrine?,"The Truman Doctrine, also known as the United ..."
3,3,Which film features a character named Jack Ske...,The Nightmare Before Christmas
4,4,What are the components of the cytoskeleton?,"The cytoskeleton, which provides the framework..."


In [47]:
test_df.shape

(1001, 3)

In [49]:
new_test_df=pd.DataFrame()

new_test_df['tokenized_Question'] = test_df['Question'].apply(spacy_tokenizer)
new_test_df['tokenized_Response'] = test_df['Response'].apply(spacy_tokenizer)

In [58]:
new_test_df.loc[new_test_df['tokenized_Response'].isnull()]

,tokenized_Question,tokenized_Response


In [57]:
new_test_df.iloc[446,1] = 'no'
new_test_df.iloc[969,1] = 'no'

In [59]:
new_test_df.head()

,tokenized_Question,tokenized_Response
0,multiply 13 7,multiply 13 7 result 64 answer question 64
1,main protagonist years war,important character war include king henry v11...
2,importance truman doctrine,truman doctrine know united states involvement...
3,film feature character jack skellington,nightmare christmas
4,component cytoskeleton,cytoskeleton provide framework cell internal s...


In [60]:
X_vec1 = vec1.transform(new_test_df.iloc[:,0])
X_vec2 = vec2.transform(new_test_df.iloc[:,1])
X_vec = hstack((X_vec1, X_vec2))

In [ ]:
#y_predict=model.predict(X_test)

In [29]:
print(X_vec[0])

  (0, 732)	1.0
  (0, 13487)	0.1829838846221926
  (0, 12933)	0.19151641307585615
  (0, 10864)	0.2896854709114378
  (0, 2660)	0.17433590117000042
  (0, 1940)	0.8516444980092042
  (0, 1330)	0.30038197683590007


In [61]:
print(model.predict_proba(X_vec[0]))

[[0.29517892 0.17037028 0.02375348 0.00277825 0.09257459 0.27618695
  0.13915753]]


In [62]:
submission = pd.read_csv('/kaggle/input/h2oai-predict-the-llm/sample_submission.csv')

for i in range(len(submission)):
    output_arr = model.predict_proba(X_vec[i])
    submission.iloc[i,1] = output_arr[0][0]
    submission.iloc[i,2] = output_arr[0][1]
    submission.iloc[i,3] = output_arr[0][2]
    submission.iloc[i,4] = output_arr[0][3]
    submission.iloc[i,5] = output_arr[0][4]
    submission.iloc[i,6] = output_arr[0][5]
    submission.iloc[i,7] = output_arr[0][6]

In [64]:
submission.to_csv('submission.csv', index=False)

In [59]:
'''
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, max_depth=5)
model.fit(X_train, y_train)

y_predict=model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.18      0.26      0.21        39
           1       0.11      0.25      0.15        24
           2       0.02      0.05      0.03        19
           3       0.72      0.34      0.46       121
           4       0.23      0.22      0.22        60
           5       0.16      0.38      0.22        24
           6       0.42      0.22      0.29       110

    accuracy                           0.26       397
   macro avg       0.26      0.24      0.23       397
weighted avg       0.40      0.26      0.30       397



In [60]:
'''
from sklearn.linear_model import SGDClassifier

model = SGDClassifier()
model.fit(X_train, y_train)

y_predict=model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.11      0.13      0.12        45
           1       0.07      0.08      0.08        49
           2       0.02      0.02      0.02        65
           3       0.46      0.33      0.39        78
           4       0.05      0.05      0.05        57
           5       0.32      0.30      0.31        60
           6       0.05      0.07      0.06        43

    accuracy                           0.15       397
   macro avg       0.15      0.14      0.15       397
weighted avg       0.17      0.15      0.16       397



In [61]:
'''
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X_train, y_train)

y_predict=model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.11      0.06      0.07       105
           1       0.25      0.36      0.29        39
           2       0.04      0.04      0.04        49
           3       0.33      0.42      0.37        45
           4       0.09      0.07      0.08        70
           5       0.12      0.27      0.17        26
           6       0.07      0.06      0.07        63

    accuracy                           0.14       397
   macro avg       0.14      0.18      0.16       397
weighted avg       0.13      0.14      0.13       397



In [62]:
'''
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()
model.fit(X_train, y_train)

y_predict=model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_predict,y_test))

              precision    recall  f1-score   support

           0       0.05      0.05      0.05        57
           1       0.04      0.03      0.03        60
           2       0.02      0.01      0.01        91
           3       0.14      0.15      0.14        54
           4       0.04      0.03      0.03        60
           5       0.18      0.29      0.22        35
           6       0.02      0.03      0.02        40

    accuracy                           0.07       397
   macro avg       0.07      0.08      0.07       397
weighted avg       0.06      0.07      0.06       397



## Another approach:

In [15]:
import numpy as np
import pandas as pd

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [17]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
dataset = pd.read_csv('/kaggle/input/h2oai-predict-the-llm/train.csv')

In [21]:
dataset.head()

,Question,Response,target
0,Explain the concept of coevolution.,Coevolution is a biological process that occur...,3
1,Is it possible that recurring fever and chills...,"Yes, recurring fever and chills can be a sympt...",4
2,Evaluate the expression 3!,The expression 3! represents the factorial of ...,1
3,What are the roles of different types of RNA i...,1. Messenger RNA (mRNA): mRNA carries genetic ...,3
4,What is the role of gene flow in population ge...,Gene flow refers to the movement of individual...,3


In [22]:
dataset.isnull().sum()

Question    0
Response    7
target      0
dtype: int64

In [23]:
dataset = dataset.fillna('')

In [9]:
dataset['content'] = dataset['Question']+' '+dataset['Response']

In [10]:
print(dataset['content'])

0       Explain the concept of coevolution. Coevolutio...
1       Is it possible that recurring fever and chills...
2       Evaluate the expression 3! The expression 3! r...
3       What are the roles of different types of RNA i...
4       What is the role of gene flow in population ge...
                              ...                        
3971    You will always find me in the past. I can be ...
3972    To give me to someone i don’t belong to is cow...
3973    Could severe abdominal pain and vomiting be in...
3974    How can I increase my productivity at work or ...
3975    Who were the leaders of the Reconquista in Spa...
Name: content, Length: 3976, dtype: object


In [24]:
port_stem = PorterStemmer()

In [25]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [34]:
dataset['content'] = dataset['content'].apply(stemming)

In [35]:
dataset['content']

0       explain concept coevolut coevolut biolog proce...
1       possibl recur fever chill could indic malaria ...
2       evalu express express repr factori product pos...
3       role differ type rna cell messeng rna mrna mrn...
4       role gene flow popul genet gene flow refer mov...
                              ...                        
3971    alway find past creat present futur never tain...
3972    give someon belong cowardli take nobl game nob...
3973    could sever abdomin pain vomit indic pancreat ...
3974    increa product work school priorit task make l...
3975    leader reconquista spain reconquista led vario...
Name: content, Length: 3976, dtype: object

In [36]:
X = dataset['content'].values
Y = dataset['target'].values

In [37]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [26]:
#using lemmatization

dataset['Question'] = dataset['Question'].apply(stemming)
dataset['Response'] = dataset['Response'].apply(stemming)

In [27]:
X1 = dataset['Question'].values
X2 = dataset['Response'].values
Y = dataset['target'].values

In [28]:
vectorizer1 = TfidfVectorizer()
vectorizer2 = TfidfVectorizer()

vectorizer1.fit(X1)
vectorizer2.fit(X2)

X1 = vectorizer1.transform(X1)
X2 = vectorizer2.transform(X2)

In [29]:
from scipy.sparse import hstack
X_data = hstack((X1, X2))

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y, test_size = 0.2, stratify=Y, random_state=2)

In [41]:
import lightgbm as ltb

model = ltb.LGBMClassifier()
model.fit(X_train, Y_train)

LGBMClassifier()

In [42]:
y_predict=model.predict(X_test)

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_predict,Y_test))

              precision    recall  f1-score   support

           0       0.27      0.33      0.30        95
           1       0.41      0.36      0.38       128
           2       0.17      0.14      0.15       134
           3       0.52      0.55      0.53       107
           4       0.24      0.22      0.23       122
           5       0.32      0.34      0.33       105
           6       0.30      0.32      0.31       105

    accuracy                           0.32       796
   macro avg       0.32      0.32      0.32       796
weighted avg       0.31      0.32      0.31       796



In [34]:
'''
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, max_depth=5)
model.fit(X_train, Y_train)

y_predict=model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_predict,Y_test))

              precision    recall  f1-score   support

           0       0.26      0.34      0.30        89
           1       0.12      0.25      0.17        56
           2       0.09      0.26      0.13        39
           3       0.76      0.34      0.47       258
           4       0.15      0.17      0.16       100
           5       0.05      0.26      0.09        23
           6       0.50      0.25      0.33       231

    accuracy                           0.28       796
   macro avg       0.28      0.27      0.23       796
weighted avg       0.46      0.28      0.32       796



In [49]:
X_data = X_data.toarray()

In [51]:
X_data.shape

(3976, 13270)

In [50]:
submission = pd.read_csv('/kaggle/input/h2oai-predict-the-llm/sample_submission.csv')

for i in range(len(submission)):
    output_arr = model.predict_proba(X_data[i])
    submission.iloc[i,1] = output_arr[0][0]
    submission.iloc[i,2] = output_arr[0][1]
    submission.iloc[i,3] = output_arr[0][2]
    submission.iloc[i,4] = output_arr[0][3]
    submission.iloc[i,5] = output_arr[0][4]
    submission.iloc[i,6] = output_arr[0][5]
    submission.iloc[i,7] = output_arr[0][6]
    
    
submission.to_csv('submission.csv', index=False)

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## Using Lemmatization:

In [ ]:
#using lemmatization

import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
#using lemmatization

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
#using lemmatization

def get_wordnet_pos(tag):
    tag = tag[0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in nltk.pos_tag(words)]
    return ' '.join(lemmatized_words)

In [ ]:
dataset['content'] = dataset['content'].apply(lemmatize_text)